# RAG 평가를 위한 합성 데이터셋 생성


RAG(검색 증강 생성) 강화 파이프라인의 성능을 평가하는 것은 매우 중요합니다.
하지만 문서에서 수백 개의 QA(질문-답변-컨텍스트) 샘플을 수동으로 만드는 것은 시간이 많이 소요되고 노동 집약적입니다. 또한 사람이 생성한 질문은 철저한 평가에 필요한 복잡성 수준에 도달하는 데 어려움이 있을 수 있어, 결과적으로 평가의 품질에 영향을 미칠 수 있습니다.

### 참조
- [Testset Generation for RAG](https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/)

---




## 0. 환경 설정

In [4]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)


module_path = ".."
add_python_path(module_path)


from libs.generate_synthesis_data import (
    PDFLoader,split_text,add_meta_into_chunk,
    create_prompt,
    converse_with_bedrock_tools,
    parse_tool_use,
    generate_qa_dataset,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock already exists
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock/notebook', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python310.zip', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/lib-dynload', '', '/home/sagemaker-user/.conda/envs/regas_bedrock/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/17_rag_evaluation_ragas_on_bedrock']


## 1. 실습에 사용된 문서

- 파일 위치: ../data/241231-Sonnet-recipe.pdf
- 이 문서는 Claude Sonnet3.5 로 생성된 데이타를 pdf 파일로 생성 했습니다.


In [5]:
loader = PDFLoader("../data/241231-Sonnet-recipe.pdf", start_page=1, end_page=3)
docs = loader.load()
docs

{'page_content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n기와 달걀을 간장 베이스의 달달한 육수에 조려 밥 위에 올립니다.\n중식으로는 마파두부가 있는데, 두부와 다진 고기를 매운 두반장 소스에 볶아 만\n듭니다. 깐풍기는 닭고기를 튀겨서 매콤달콤한 소스를 絡めて만드는 요리이고, 탕\n수육은 돼지고기를 튀겨서 새콤달콤한 소스를 부어 먹습니다.\n태국 요리로는 팟타이가 유명한데, 쌀국수를 새우와 함께 볶고 땅콩가루를 뿌려\n고소한 맛을 냅니다. 그린커리는 코코넛밀크와 향신료를 넣어 만든 커리에 닭고기\n와 가지를 넣어 끓입니다. 톰얌쿵은 새우와 버섯을 레몬그라스, 카피르라임 등의\n향신료와 함께 시큼매콤하게 끓이는 수프입니다.\n베트남 요리에서는 쌀국수인 포가 대표적인데, 소고기 육수에 쌀국수와 고기를 넣\n고 고수, 숙주나물을 곁들여 먹습니다. 반미는 바게트빵에 각종 채소와 고기를\n넣어 만드는 샌드위치이며, 분짜는 구운 돼지고기와 쌀국수를 느억맘 소스에 찍어\n먹는 요리입니다.\n이탈리아 요리로 더 들어가보면, 리

## 2.  문서를 Chunk 로 분리 및 메타 데이터 삽입

In [6]:
chunks = split_text(
                text = docs['page_content'], 
                chunk_size=512, 
                chunk_overlap=0)
print("length of chunks: ", len(chunks))
chunks[0]

length of chunks:  8


'볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n'

In [7]:
chunks_with_metadata = add_meta_into_chunk(chunks, docs)    
chunks_with_metadata[0]

{'content': '볶음요리로는 먼저 김치볶음밥이 있는데, 잘 익은 김치를 잘게 썰어 밥과 함께\n볶다가 마지막에 참기름을 둘러 고소한 향을 더합니다. 제육볶음은 돼지고기를 고\n추장 양념에 버무려 매콤달콤하게 볶아내며, 양파와 당근 등 채소를 함께 넣어\n영양을 높입니다. 낙지볶음은 신선한 낙지를 손질해 고추장 양념에 볶아 매콤하게\n만드는 요리입니다.\n양식으로는 크림파스타가 대표적인데, 생크림과 마늘, 양파를 볶아 소스를 만들고\n베이컨이나 새우를 더해 고소하게 완성합니다. 토마토파스타는 토마토소스에 마늘\n과 양파를 볶아 넣고 바질을 곁들여 상큼한 맛을 냅니다. 미트소스스파게티는 다\n진 쇠고기를 토마토소스와 함께 오래 끓여 깊은 맛을 만듭니다.\n일식에서는 돈카츠가 인기 있는 메뉴로, 돼지고기를 얇게 펴서 빵가루를 묻혀 바\n삭하게 튀겨내고 특제 소스를 곁들입니다. 카레라이스는 당근, 감자, 양파를 카\n레가루와 함께 끓여 걸쭉한 소스를 만들어 밥에 부어 먹습니다. 오야코동은 닭고\n',
 'metadata': {'chunk_id': 0, 'filename': '../data/241231-Sonnet-recipe.pdf'}}

## 3. Test Q&A Dataset Generation

### 3.1. Amazon Bedrock 사용위한 설정

In [8]:
import boto3
from botocore.config import Config

region = 'us-west-2'
retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

### 3.2. 파라미터 설정 및 생성

In [11]:
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
num_of_generated_sample = 10
output_file="../data/sample_qa_dataset.jsonl"

In [12]:
%%time

generate_qa_dataset(
                    model_id = model_id, 
                    chunks = chunks_with_metadata, 
                    num_pairs = num_of_generated_sample, 
                    boto3_client = boto3_client,
                    output_file= output_file,
                    # verbose=True,
                    verbose=False,
)

{'question': '중동, 한식, 일식, 중식, 동남아, 서양 요리 중에서 병아리콩을 주재료로 사용하는 요리들을 비교하고, 이 요리들의 조리법과 문화적 특징을 설명해주세요.', 'ground_truth': '병아리콩을 주재료로 사용하는 요리는 주로 중동 요리에서 찾아볼 수 있습니다. 대표적으로 팔라펠과 후무스가 있습니다.\n\n팔라펠은 병아리콩으로 만든 완자를 튀겨서 피타빵에 채소와 함께 넣어 먹는 중동 요리입니다. 병아리콩을 으깨어 향신료와 함께 반죽한 후 동그랗게 빚어 튀겨내는 방식으로 만듭니다. 이는 중동 지역의 길거리 음식으로 널리 사랑받고 있으며, 채식주의자들에게도 좋은 단백질 공급원이 됩니다.\n\n후무스는 병아리콩을 갈아서 타히니(참깨 페이스트), 레몬즙, 올리브오일과 함께 섞어 만드는 디핑소스입니다. 부드러운 크림 같은 질감이 특징이며, 피타빵이나 야채 스틱과 함께 즐깁니다. 후무스는 중동 지역의 메제(전채요리)로 자주 등장하며, 건강에 좋은 지방과 단백질이 풍부합니다.\n\n제시된 다른 요리 문화권(한식, 일식, 중식, 동남아, 서양 요리)에서는 병아리콩을 주재료로 사용하는 대표적인 요리가 언급되지 않았습니다. 이는 병아리콩이 주로 중동과 지중해, 인도 지역에서 많이 재배되고 소비되는 식재료이기 때문입니다. 다만 최근에는 건강식에 대한 관심이 높아지면서 다양한 문화권에서 병아리콩을 활용한 퓨전 요리들이 개발되고 있습니다.', 'question_type': 'complex', 'contexts': '중동 요리에서는 샤워마가 유명한데, 양고기나 닭고기를 특제 향신료에 재워 수직\n으로 구워 얇게 썰어 피타빵에 싸먹습니다. 팔라펠은 병아리콩으로 만든 완자를\n튀겨서 피타빵에 채소와 함께 넣어 먹습니다. 후무스는 병아리콩을 갈아서 타히\n니, 레몬즙, 올리브오일과 함께 섞어 만드는 디핑소스입니다.\n한식으로 돌아와서, 갈비찜은 소갈비를 간장 베이스의 달달한 양념으로 푹 조려내\n는 요리입니다. 닭갈비는 닭고기를 고추장 양념에 채소와 함께 볶아 치

[{'question': '중동, 한식, 일식, 중식, 동남아, 서양 요리 중에서 병아리콩을 주재료로 사용하는 요리들을 비교하고, 이 요리들의 조리법과 문화적 특징을 설명해주세요.',
  'ground_truth': '병아리콩을 주재료로 사용하는 요리는 주로 중동 요리에서 찾아볼 수 있습니다. 대표적으로 팔라펠과 후무스가 있습니다.\n\n팔라펠은 병아리콩으로 만든 완자를 튀겨서 피타빵에 채소와 함께 넣어 먹는 중동 요리입니다. 병아리콩을 으깨어 향신료와 함께 반죽한 후 동그랗게 빚어 튀겨내는 방식으로 만듭니다. 이는 중동 지역의 길거리 음식으로 널리 사랑받고 있으며, 채식주의자들에게도 좋은 단백질 공급원이 됩니다.\n\n후무스는 병아리콩을 갈아서 타히니(참깨 페이스트), 레몬즙, 올리브오일과 함께 섞어 만드는 디핑소스입니다. 부드러운 크림 같은 질감이 특징이며, 피타빵이나 야채 스틱과 함께 즐깁니다. 후무스는 중동 지역의 메제(전채요리)로 자주 등장하며, 건강에 좋은 지방과 단백질이 풍부합니다.\n\n제시된 다른 요리 문화권(한식, 일식, 중식, 동남아, 서양 요리)에서는 병아리콩을 주재료로 사용하는 대표적인 요리가 언급되지 않았습니다. 이는 병아리콩이 주로 중동과 지중해, 인도 지역에서 많이 재배되고 소비되는 식재료이기 때문입니다. 다만 최근에는 건강식에 대한 관심이 높아지면서 다양한 문화권에서 병아리콩을 활용한 퓨전 요리들이 개발되고 있습니다.',
  'question_type': 'complex',
  'contexts': '중동 요리에서는 샤워마가 유명한데, 양고기나 닭고기를 특제 향신료에 재워 수직\n으로 구워 얇게 썰어 피타빵에 싸먹습니다. 팔라펠은 병아리콩으로 만든 완자를\n튀겨서 피타빵에 채소와 함께 넣어 먹습니다. 후무스는 병아리콩을 갈아서 타히\n니, 레몬즙, 올리브오일과 함께 섞어 만드는 디핑소스입니다.\n한식으로 돌아와서, 갈비찜은 소갈비를 간장 베이스의 달달한 양념으로 푹 조려내\n는 요리입니다. 닭갈비는 닭고기를 고추장 양념에 채소와 